In [ ]:
#Runnable Interface lets you plug your own Python logic into a LangChain pipeline just like an LLM, tool, or chain.
#RunnableLambda = “wrap a Python function so it can live inside a LangChain pipeline.”
#LLMs, prompts, agents → all are Runnables.

In [1]:
from dotenv import load_dotenv
import os 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_tavily import TavilySearch
from langchain.tools import tool
from pydantic import BaseModel , Field 
from langchain_classic.agents.react.agent import create_react_agent
from langchain_classic.agents import AgentExecutor

from langchain_classic import hub #Think of it like GitHub, but specifically for LangChain prompts and chains.
#so this is a place where you can find structured prompts and chains 

load_dotenv(override=True)


True

In [2]:
from typing import List
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

In [3]:
class Source(BaseModel):
    """Schema for a source used by the agent"""

    url : str = Field(description="The url of the source")

class AgentResponse(BaseModel):
    """Scehama for agent response with answer and sources"""

    answer : str = Field(description="Agent's answer to query")
    sources : List[Source] = Field(
        default_factory=list , description="List of sources used by the agent to answer the query"
    )

In [4]:
REACT_PROMPT_WITH_FORMAT_INSTRUCTIONS = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

IMPORTANT:
You must respond with exactly ONE of the following:
- An Action and Action Input
- OR a Final Answer
Never both in the same response.

"""

In [5]:
output_parser = PydanticOutputParser(pydantic_object=AgentResponse)

In [6]:
react_prompt_with_format_instructions = PromptTemplate(
    template = REACT_PROMPT_WITH_FORMAT_INSTRUCTIONS,
    input_variables = ["input" , "agent_scratchpad" , "tools" , "tool_names" , "format_instructions"]
).partial(format_instructions = output_parser.get_format_instructions())
#“The output must be a valid JSON object with the following fields…”
#format_instructions = rules that tell the LLM how its FINAL output must be structured so the output parser can parse it

In [10]:
import re 
tools = [TavilySearch()]
llm = ChatOpenAI(model="openai/gpt-4o-mini", temperature=0)
agent = create_react_agent(
    llm, 
    tools,  
    react_prompt_with_format_instructions,
)

agent_executor = AgentExecutor(agent = agent , tools = tools , verbose = True )
extract_output = RunnableLambda(lambda x : x["output"])
parse_output = RunnableLambda(lambda x : output_parser.parse(x))
chain = agent_executor | extract_output | parse_output
#lambda is an anonymous function that takes an input and returns an output
#extract_output is a lambda function that takes an input and returns the output
#parsed_output is a lambda function that takes an input and returns the parsed output
#chain is a pipeline that takes an input, extracts the output, and parses the output

In [11]:
result = chain.invoke(
    input={
        "input":"search for 3 Ai jobs in india"
    }
) 



> Entering new AgentExecutor chain...
Action: tavily_search  
Action Input: "AI jobs in India"  {'query': 'AI jobs in India', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.linkedin.com/jobs/search/?currentJobId=3916826170&f_TPR=r86400&geoId=102713980&keywords=artificial%20intelligence&location=India&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true', 'title': '3,000+ Artificial Intelligence jobs in India - LinkedIn', 'content': '3,000+ Artificial Intelligence Jobs in India · Data Scientist · Business Analyst · AI Software Engineer · Python AI · Generative AI, AI/ML, Python, Databricks.', 'score': 0.91988057, 'raw_content': None}, {'url': 'https://in.linkedin.com/jobs/artificial-intelligence-jobs', 'title': '41000+ Artificial Intelligence jobs in India - Bengaluru - LinkedIn', 'content': '41,000+ Artificial Intelligence Jobs in India · Data Scientist · AI/ML · Python ML developer · Artificial Intelligence Specialist · Python ML · Software'

OutputParserException: Invalid json output: Here are three AI job listings in India:

1. **Data Scientist** - LinkedIn: [3,000+ Artificial Intelligence jobs in India](https://www.linkedin.com/jobs/search/?currentJobId=3916826170&f_TPR=r86400&geoId=102713980&keywords=artificial%20intelligence&location=India&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true)

2. **AI/ML Developer** - LinkedIn: [41,000+ Artificial Intelligence jobs in India](https://in.linkedin.com/jobs/artificial-intelligence-jobs)

3. **Data Scientist / Senior Data Scientist** - Indeed: [6,000 Artificial Intelligence Job Vacancies](https://in.indeed.com/q-artificial-intelligence-jobs.html)
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 